In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install git+https://github.com/theislab/drvi.git@main

In [ ]:
pip install gprofiler

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
import anndata as ad
import scanpy as sc
from matplotlib import pyplot as plt
from IPython.display import display
from gprofiler import gprofiler
import torch
import numpy as np
import drvi
from drvi.model import DRVI
from drvi.utils.misc import hvg_batch

In [ ]:
sc.settings.set_figure_params(dpi=100, frameon=False)
sc.set_figure_params(dpi=100)
sc.set_figure_params(figsize=(3, 3))
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.figsize"] = (3, 3)

In [ ]:
adata = sc.read("/work/DRVI/fixed_data/Focal_cortical/focal_cortical_processed_RAW.h5ad")
adata

In [ ]:
adata.obs["cell_type"]

In [ ]:
# Subset the data
subset_mask = (adata.obs['sex'] == 'male') & (adata.obs['cell_type'] == 'microglial cell')
subset_adata = adata[subset_mask].copy()  
train_adata = adata[~subset_mask].copy()  # Training data is everything except the subset

In [ ]:
train_adata.layers["counts"] = train_adata.X.copy()

In [ ]:
train_adata.layers["counts"]

In [ ]:
print(type(train_adata.layers["counts"]))
print(train_adata.layers["counts"].shape)


In [ ]:
print(train_adata.layers["counts"][:50, :50])  # View top-left 5×5 corner

In [ ]:
import numpy as np

# Get the 50×50 submatrix
submatrix = train_adata.layers["counts"][:50, :50]

# Convert to COO format to access .data, .row, .col
coo = submatrix.tocoo()

# Sort by value descending
sorted_indices = np.argsort(-coo.data)  # negative for descending sort

# Print sorted values with their coordinates
for i in sorted_indices:
    print(f"({coo.row[i]}, {coo.col[i]}) -> {coo.data[i]}")

In [ ]:
# Setup data
DRVI.setup_anndata(
    train_adata,
    # DRVI accepts count data by default.
    # Do not forget to change gene_likelihood if you provide a non-count data.
    layer="counts",
    # Always provide a list. DRVI can accept multiple covariates.
    categorical_covariate_keys=["cell_type", "sex", "donor_id", "development_stage_ontology_term_id", "lateralization", 
                                "tissue"],
    # DRVI accepts count data by default.
    # Set to false if you provide log-normalized data and use normal distribution (mse loss).
    is_count_data=True,
)

# construct the model
model = DRVI(
    train_adata,
    # Provide categorical covariates keys once again. Refer to advanced usages for more options.
    categorical_covariates=["cell_type", "sex", "donor_id", "development_stage_ontology_term_id", "lateralization", 
                                "tissue"],
    n_latent=128,
    # For encoder and decoder dims, provide a list of integers.
    encoder_dims=[128, 128],
    decoder_dims=[128, 128],
)
model

In [ ]:
# For cpu training you should add the following line to the model.train parameters:
#accelerator="cpu", devices=1,
#
# For mps acceleration on macbooks, add the following line to the model.train parameters:
# accelerator="mps", devices=1,
#
# For gpu training don't provide any additional parameter.
# More details here: https://lightning.ai/docs/pytorch/stable/accelerators/gpu_basic.html

n_epochs = 400

# train the model
model.train(
    max_epochs=n_epochs,
    early_stopping=False,
    early_stopping_patience=20,
    # mps
    # accelerator="mps", devices=1,
    # cpu
    #accelerator="cpu", devices=1,
    # gpu: no additional parameter
    #
    # No need to provide `plan_kwargs` if n_epochs >= 400.
    plan_kwargs={
        "n_epochs_kl_warmup": n_epochs,
    },
)

# Runtime:
# The runtime for CPU laptop (M1) is 208 minutes
# The runtime for Macbook gpu (M1) is 64 minutes
# The runtime for GPU (A100) is 17 minutes

In [ ]:
model.save("DRVI/LOA_DRVI/DRVI/LOA_DRVI/Focal_cortical_LOA_fixed_data_1005")

In [ ]:
model = model.load("/work/DRVI/LOA_DRVI/DRVI/LOA_DRVI/DRVI/LOA_DRVI/Focal_cortical_LOA_fixed_data_1005", adata = train_adata)

In [ ]:
model

In [ ]:
def predict(model, adata):
    model._validate_anndata(adata)
    model.module.eval()

    scdl = model._make_data_loader(adata=adata, indices=None, batch_size=128, shuffle=False)
    mus = []
    for tensors in scdl:
        inference_outputs, generative_outputs = model.module.forward(
                    tensors,
                    compute_loss=False,
                )
        _mus = torch.nan_to_num(generative_outputs['px'].mean, nan=0, neginf=0, posinf=100) 
        mus.append(_mus.detach().cpu().numpy())
    mus = np.concatenate(mus, axis=0)
    out_adata = adata.copy()
    out_adata.X = mus
    return out_adata


#model._validate_anndata(subset_adata)
#rec = predict(model, subset_adata)

In [ ]:
female_micro = train_adata[
    (train_adata.obs["sex"] == "female") &
    (train_adata.obs["cell_type"] == "microglial cell")
].copy()

In [ ]:
rec_female = predict(model, female_micro)

In [ ]:
cf = female_micro.copy()
cf.obs["sex"] = "male"

In [ ]:
rec_cf = predict(model, cf)

In [ ]:
# Compute per-gene ground-truth means
# Gt for females is now females with hepatocytes

female_gt = (
    female_micro.X.toarray().mean(axis=0)
    if hasattr(female_micro.X, "toarray")
    else female_micro.X.mean(axis=0)
)


#    Gt for the male is now the held out dataset
male_gt = (
    subset_adata.X.toarray().mean(axis=0)
    if hasattr(subset_adata.X, "toarray")
    else subset_adata.X.mean(axis=0)
)

In [ ]:
import numpy as np

In [ ]:
#  Compute per-gene prediction means 
baseline_pred = np.asarray(rec_female.X.mean(axis=0)).ravel()
cf_pred       = np.asarray(rec_cf.X.mean(axis=0)).ravel()

In [ ]:
f_m = np.sqrt(np.mean((baseline_pred - male_gt)**2)) #pred-F --- TM
m_m = np.sqrt(np.mean((cf_pred - male_gt)**2)) # pred_M --- TM
f_f  =np.sqrt(np.mean((baseline_pred - female_gt)**2)) # pref F ----- TF
m_f = np.sqrt(np.mean((cf_pred - female_gt)**2)) # pref M ---- TF

print(f"RMSE pred F --- TM: {f_m:.4f}")
print(f"RMSE pred M --- TM (counterfactual)  : {m_m:.4f}")
print(f"RMSE pred F ----- TF : {f_f:.4f}")
print(f"RMSE pred M ---- TF (counterfactual)  : {m_f:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Define RMSE values
rmse_data = [
    ["0.0172", "0.0587"],  # True Female
    ["0.2061", "0.2092"],  # True Male
]

# Define labels
column_labels = ["Pred Female", "Pred Male (CF)"]
row_labels = ["True Female", "True Male"]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 2))
ax.axis('tight')
ax.axis('off')

# Create the table
table = ax.table(
    cellText=rmse_data,
    rowLabels=row_labels,
    colLabels=column_labels,
    cellLoc='center',
    loc='center'
)

table.scale(1, 2)  # Increase row height
table.auto_set_font_size(False)
table.set_fontsize(12)

# Color the lower row cells
# Note: Rows and columns are 1-indexed in table.get_celld()
table[(2, 0)].set_facecolor("orange")  # True Male, Pred Female
table[(2, 1)].set_facecolor("lightblue")  # True Male, Pred Male

plt.title("RMSE between Predictions and Ground truth", pad=20)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Log-transform the data
log_male_gt = np.log1p(male_gt)
log_cf_pred = np.log1p(cf_pred)
log_baseline_pred = np.log1p(baseline_pred)

# Scatter plot of log-transformed ground truth vs. predictions
fig, ax = plt.subplots(figsize=(10, 6))

sns.scatterplot(
    x=log_male_gt, y=log_cf_pred,
    alpha=0.7, ax=ax
)

sns.scatterplot(
    x=log_male_gt, y=log_baseline_pred,
    alpha=0.7, ax=ax
)

# Identity line (in log space)
mn = min(log_male_gt.min(), log_male_gt.min())
mx = max(log_male_gt.max(), log_male_gt.max())
ax.plot([mn, mx], [mn, mx], ls="--", color="red")


ax.set_xlabel("Log Ground Truth (Male)")
ax.set_ylabel("Log Predicted Mean Expression")
ax.set_title("Leave One Out analysis" )
ax.legend(loc="upper left")
plt.tight_layout()
ax.grid(False)
plt.show()
